## Notebook principal
<br>
Implémentation du modèle (fonctions objectifs, contraintes et résolution/optimisation).

### Importations

In [1]:
import gurobipy as gp # solver
from gurobipy import GRB
import numpy as np

In [2]:
from InstanceClass import Instance # custom class built to load and encode .json files (instances)

### Loading the data

In [3]:
try:
    instance = Instance(path="toy_instance.json")
    instance.build_instance()
except FileNotFoundError:
    instance = Instance(path="data/toy_instance.json")
    instance.build_instance()

In [4]:
instance.variables # check that everything works as expected

{'NP': 5,
 'NC': 3,
 'NA': 3,
 'H': 5,
 'GAIN': [20, 15, 15, 20, 10],
 'PENALTIES': array([[0., 0., 0., 3., 6.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 3.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 0.]]),
 'STAFF_QUALIFICATIONS': array([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 1.]]),
 'COST_PROJECT': array([[1., 1., 1.],
        [1., 2., 0.],
        [1., 0., 2.],
        [0., 2., 1.],
        [0., 0., 2.]]),
 'CONGES': array([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])}

### Création de la matrice des affectations
<br>
On définit la matrice A des affections des collaborateurs sur les projets (de taille n_c ∗ h  ∗ n_p ∗ n): 
<br>
$
𝐴 = {𝑎_{𝑖,𝑗,𝑘,l}}
$
<br>


On en extrait une matrice 𝐴𝑝 = {𝑎𝑝𝑗,𝑘} indiquant si à l’instant j le projet vient d’être
rendu 


In [5]:
I=3
J=5
L=5
K=3

In [6]:
def solution_creation(variables):
    shape=(variables["NC"],variables["H"],variables["NP"],variables["NA"])
    return shape

def calcul_work(Affectation):
    rep = [np.sum([Affectation[i][j] for i in range(3)],axis=0) for j in range(np.shape(Affectation)[1])]
    return np.array(rep)

def calcul_temporel_work(work,horizon,nb_projet):
    rep = [[gp.quicksum(work[0:i])[j] for i in range(1,horizon+1)] for j in range(nb_projet)]
    return np.array(rep)

def calcul_work_final(work):
    rep=gp.quicksum(work)  
    return rep

def calcul_Ap(rep,cost_projet):
    X = [[(rep[i][j] >= cost_projet[i]) for j in range(len(rep))] for i in range(len(cost_projet))]
    return np.array(X)

In [7]:
# données propres au problème que l'on va modéliser
shape = solution_creation(instance.variables)
GAIN = instance.variables["GAIN"]
PENALTIES = instance.variables["PENALTIES"]
COST_PROJECT = instance.variables["COST_PROJECT"]
STAFF_QUALIFICATIONS = instance.variables["STAFF_QUALIFICATIONS"]
CONGES = instance.variables["CONGES"]
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]


In [8]:
BENEFICES = np.array([GAIN[i]-PENALTIES[i] for i in range(L)])

In [9]:
MAX_COST = np.max(COST_PROJECT)

### Instanciation du modèle

In [10]:
model = gp.Model("matrix1") # instanciation du modèle

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29


### Définition des variables

In [11]:
# Create variables
Affectation = model.addMVar(shape=(I,J,L,K), vtype=GRB.BINARY, name="Affectation") # matrice personnes/jours/projet/compétence (tous les projets)
Done_Project = model.addMVar(shape=(L,J), vtype=GRB.BINARY, name="Done_Project") # matrice projets/temps (tous les projets)


In [12]:
z_fo2 = model.addVar(vtype=GRB.INTEGER,name="Max_Projet")
delta = model.addMVar(shape=(L,I),vtype=GRB.INTEGER,name="Contrainte2")

In [13]:
z_fo3 = model.addVar(vtype=GRB.INTEGER,name="Duree_Max_Projet")
Begin_Project = model.addMVar(shape=(L,J), vtype=GRB.BINARY, name="Begin_Project") # matrice projets/temps (tous les projets)
u = np.arange(J)+1

In [14]:
Ap = calcul_Ap(calcul_work(Affectation),COST_PROJECT) # pas utilisé pour le moment
final_work = calcul_work_final(calcul_work(Affectation)) # somme des travaux menés par projet (sert à vérifier qu'un projet est complété)
# dimension de final_work : matrice projets/compétences (somme sur les employés par compétence et par projet, au temps final)
temp_work = calcul_temporel_work(calcul_work(Affectation),instance.variables["H"],L) # somme cumulée dans le temps du travail effectué par projet et 
# par compétence (on veut l'utiliser plutôt que final_work à terme) ; temp_work[-1] = finl_work

### Définition des fonctions objectifs

In [15]:
# définition des objectifs
model.setObjective(gp.quicksum(gp.quicksum(Done_Project*BENEFICES)), GRB.MAXIMIZE)
#model.setObjective(z_fo2, GRB.MINIMIZE)
#model.setObjective(z_fo3, GRB.MINIMIZE)


### Définition des contraintes

In [16]:
model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1,1,1]))) <= 1-CONGES[i][j] for i in range(I) for j in range(J) for l in range(L)), name="Day_off_Constraint") 
model.addConstrs((temp_work[i][j] >= COST_PROJECT[i]*Done_Project[i][j] for i in range(L) for j in range(J)), name="Done") 
model.addConstrs((gp.quicksum(Done_Project[i]) <= 1 for i in range(L)), name="Only_Done_One_Time") 

{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>}

In [17]:
epsilon=0.0001

In [18]:
model.addConstrs((gp.quicksum(delta)[i] <= z_fo2 for i in range(I)), name="C2.1") 
model.addConstrs(((J*(1-delta[l][i]))<=gp.quicksum(gp.quicksum(Affectation[i])[l]) for i in range(I) for l in range(L)), name="C2.2")
model.addConstrs((gp.quicksum(gp.quicksum(Affectation[i])[l])+epsilon<=(J*delta[l][i]) for i in range(I) for l in range(L)), name="C2.3") 

{(0, 0): <MConstr () *awaiting model update*>,
 (0, 1): <MConstr () *awaiting model update*>,
 (0, 2): <MConstr () *awaiting model update*>,
 (0, 3): <MConstr () *awaiting model update*>,
 (0, 4): <MConstr () *awaiting model update*>,
 (1, 0): <MConstr () *awaiting model update*>,
 (1, 1): <MConstr () *awaiting model update*>,
 (1, 2): <MConstr () *awaiting model update*>,
 (1, 3): <MConstr () *awaiting model update*>,
 (1, 4): <MConstr () *awaiting model update*>,
 (2, 0): <MConstr () *awaiting model update*>,
 (2, 1): <MConstr () *awaiting model update*>,
 (2, 2): <MConstr () *awaiting model update*>,
 (2, 3): <MConstr () *awaiting model update*>,
 (2, 4): <MConstr () *awaiting model update*>}

In [19]:
model.addConstrs((gp.quicksum(Begin_Project[i])<= 1 for i in range(L)), name="Only_Begin_One_Time") 
model.addConstrs((gp.quicksum(Begin_Project[i]) == gp.quicksum(Done_Project[i]) for i in range(L)), name="Begin_Done") 
model.addConstrs((gp.quicksum((Done_Project[i]-Begin_Project[i])*u)+1<=z_fo3 for i in range(L)), name="Only_Begin_One_Time") 


{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>}

In [20]:
model.addConstrs((temp_work[i][j][k]/MAX_COST <= gp.quicksum(Begin_Project[i][0:(j+1)]) for i in range(L) for j in range(J) for k in range(K)), name="Cant_begin_without_work") 


{(0, 0, 0): <MConstr () *awaiting model update*>,
 (0, 0, 1): <MConstr () *awaiting model update*>,
 (0, 0, 2): <MConstr () *awaiting model update*>,
 (0, 1, 0): <MConstr () *awaiting model update*>,
 (0, 1, 1): <MConstr () *awaiting model update*>,
 (0, 1, 2): <MConstr () *awaiting model update*>,
 (0, 2, 0): <MConstr () *awaiting model update*>,
 (0, 2, 1): <MConstr () *awaiting model update*>,
 (0, 2, 2): <MConstr () *awaiting model update*>,
 (0, 3, 0): <MConstr () *awaiting model update*>,
 (0, 3, 1): <MConstr () *awaiting model update*>,
 (0, 3, 2): <MConstr () *awaiting model update*>,
 (0, 4, 0): <MConstr () *awaiting model update*>,
 (0, 4, 1): <MConstr () *awaiting model update*>,
 (0, 4, 2): <MConstr () *awaiting model update*>,
 (1, 0, 0): <MConstr () *awaiting model update*>,
 (1, 0, 1): <MConstr () *awaiting model update*>,
 (1, 0, 2): <MConstr () *awaiting model update*>,
 (1, 1, 0): <MConstr () *awaiting model update*>,
 (1, 1, 1): <MConstr () *awaiting model update*>,


In [21]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 503 rows, 292 columns and 3628 nonzeros
Model fingerprint: 0x5e3af9d2
Variable types: 0 continuous, 292 integer (275 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 430 rows and 176 columns
Presolve time: 0.02s
Presolved: 73 rows, 116 columns, 512 nonzeros
Variable types: 0 continuous, 116 integer (116 binary)

Root relaxation: objective 8.000000e+01, 48 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   80.0000

### Vérification du respect des contraintes

In [22]:
Affectation_solution = Affectation.X
Done_Project_solution = Done_Project.X
Begin_Project_solution = Begin_Project.X

In [23]:
np.shape(Affectation_solution)

(3, 5, 5, 3)

In [24]:
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]

In [25]:
# Qualifications
for i in range(I):
    for j in range(J):
        for l in range(L):
            for k in range(K):
                assert Affectation_solution[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] 


In [26]:
# One day One work
for i in range(I):
    for j in range(J):
        assert np.sum(Affectation_solution[i][j])<= 1- CONGES[i][j]

In [27]:
# Projet Done one time
assert (np.sum(Done_Project_solution,axis=1)<=1).all()

In [28]:
# Assert Project done

In [29]:
instance.variables

{'NP': 5,
 'NC': 3,
 'NA': 3,
 'H': 5,
 'GAIN': [20, 15, 15, 20, 10],
 'PENALTIES': array([[0., 0., 0., 3., 6.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 3.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 0.]]),
 'STAFF_QUALIFICATIONS': array([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 1.]]),
 'COST_PROJECT': array([[1., 1., 1.],
        [1., 2., 0.],
        [1., 0., 2.],
        [0., 2., 1.],
        [0., 0., 2.]]),
 'CONGES': array([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])}

In [30]:
Done_Project_solution

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [31]:
Begin_Project_solution

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [32]:
CONGES

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [33]:
BENEFICES

array([[20., 20., 20., 17., 14.],
       [15., 15., 15., 12.,  9.],
       [15., 15., 15., 15., 12.],
       [20., 20., 20., 17., 14.],
       [10., 10., 10., 10., 10.]])

In [34]:
Begin_Project_solution

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [35]:
np.sum(Affectation_solution[0][0])

1.0

In [36]:
gp.quicksum(Affectation[0][0] @ np.transpose(np.array([1,1,1])))

<MLinExpr ()>
array( Affectation[0,0,0,0] + Affectation[0,0,0,1] + Affectation[0,0,0,2] + Affectation[0,0,1,0] + Affectation[0,0,1,1] + Affectation[0,0,1,2] + Affectation[0,0,2,0] + Affectation[0,0,2,1] + Affectation[0,0,2,2] + Affectation[0,0,3,0] + Affectation[0,0,3,1] + Affectation[0,0,3,2] + Affectation[0,0,4,0] + Affectation[0,0,4,1] + Affectation[0,0,4,2])

In [37]:
Affectation[0][0]

<MVar (5, 3)>
array([[<gurobi.Var Affectation[0,0,0,0] (value 1.0)>,
        <gurobi.Var Affectation[0,0,0,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,0,2] (value 0.0)>],
       [<gurobi.Var Affectation[0,0,1,0] (value 0.0)>,
        <gurobi.Var Affectation[0,0,1,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,1,2] (value 0.0)>],
       [<gurobi.Var Affectation[0,0,2,0] (value 0.0)>,
        <gurobi.Var Affectation[0,0,2,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,2,2] (value 0.0)>],
       [<gurobi.Var Affectation[0,0,3,0] (value 0.0)>,
        <gurobi.Var Affectation[0,0,3,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,3,2] (value 0.0)>],
       [<gurobi.Var Affectation[0,0,4,0] (value 0.0)>,
        <gurobi.Var Affectation[0,0,4,1] (value 0.0)>,
        <gurobi.Var Affectation[0,0,4,2] (value 0.0)>]])

In [38]:
i = 2

In [39]:
COST_PROJECT[i]*Done_Project[i]

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [ ]:
COST_PROJECT[i]*Done_Project[i]

In [ ]:
Affectation[1]

In [ ]:
for v in model.getVars():
    print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % model.ObjVal)